In [1]:
import tensorflow_hub as hub
import tensorflow_text as tf_text

2022-12-12 02:18:16.541890: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 02:18:17.269641: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 02:18:17.269675: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-12 02:18:19.136911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
# tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
tfhub_handle_preprocess = "tfhub_models/bert_en_uncased_preprocess"
# tfhub_handle_encoder = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
tfhub_handle_encoder = "tfhub_models/bert_en_uncased_L-12_H-768_A-12"

In [3]:
bert_preprocessor = hub.KerasLayer(tfhub_handle_preprocess)

2022-12-12 02:18:21.251586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-12 02:18:21.252313: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-12 02:18:21.252394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (djimadja): /proc/driver/nvidia/version does not exist
2022-12-12 02:18:21.253061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
text_test = ['this is such an amazing movie!']
text_preprocessed = bert_preprocessor(text_test)

print(f'Keys       : {list(text_preprocessed.keys())}')
print(f'Shape      : {text_preprocessed["input_word_ids"].shape}')
print(f'Word Ids   : {text_preprocessed["input_word_ids"][0, :12]}')
print(f'Input Mask : {text_preprocessed["input_mask"][0, :12]}')
print(f'Type Ids   : {text_preprocessed["input_type_ids"][0, :12]}')

Keys       : ['input_mask', 'input_type_ids', 'input_word_ids']
Shape      : (1, 128)
Word Ids   : [ 101 2023 2003 2107 2019 6429 3185  999  102    0    0    0]
Input Mask : [1 1 1 1 1 1 1 1 1 0 0 0]
Type Ids   : [0 0 0 0 0 0 0 0 0 0 0 0]


In [5]:
bert_encoder = hub.KerasLayer(tfhub_handle_encoder)

2022-12-12 02:18:31.968367: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 93763584 exceeds 10% of free system memory.


In [6]:
bert_results = bert_encoder(text_preprocessed)

print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Loaded BERT: tfhub_models/bert_en_uncased_L-12_H-768_A-12
Pooled Outputs Shape:(1, 768)
Pooled Outputs Values:[-0.92169875 -0.39353448 -0.5393166   0.6825623   0.43848425 -0.14021152
  0.87747115  0.26043344 -0.63112926 -0.9999658  -0.2631999   0.8510528 ]
Sequence Outputs Shape:(1, 128, 768)
Sequence Outputs Values:[[ 0.19451588  0.25141683  0.19075048 ... -0.24845086  0.3856856
   0.1329101 ]
 [-0.5947865  -0.39420336  0.2524569  ... -0.7694673   1.1564162
   0.32475716]
 [ 0.0064151  -0.15766409  0.5461024  ... -0.17451069  0.60289675
   0.42672223]
 ...
 [ 0.2194835  -0.20927066  0.5386831  ...  0.24693537  0.18250984
  -0.44427064]
 [ 0.01080252 -0.44553122  0.35990965 ...  0.31722802  0.23562811
  -0.63070595]
 [ 0.29321158 -0.10581908  0.6114752  ...  0.20745802  0.1449466
  -0.3535337 ]]


In [7]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocessor(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8435169 , -0.51327264, -0.88845724, ..., -0.74748874,
        -0.75314736,  0.91964483],
       [-0.8720836 , -0.5054398 , -0.9444668 , ..., -0.85847515,
        -0.7174535 ,  0.88082975]], dtype=float32)>

In [8]:
import pandas as pd

In [11]:
papers = pd.read_csv('datasets/ai_papers_00-17.csv')
papers.shape

(18145, 5)

In [12]:
papers.head()

,paper_title,num_of_citations,conference,year,is_link
0,Value ordering for finding all solutions,44,IJCAI,2005,True
1,A Conceptual Graph Approach for the Generation...,31,IJCAI,2007,True
2,Exploiting image contents in web search,34,IJCAI,2007,True
3,Best-First Utility-Guided Search,31,IJCAI,2007,True
4,Ambiguous part-of-speech tagging for improving...,30,IJCAI,2007,True


In [13]:
CUT_OFF = papers['num_of_citations'].median()
print(f"Median is {CUT_OFF}")
CUT_OFF = int(CUT_OFF)
# CUT_OFF = 15
print(f"Cut off used is {CUT_OFF}")

Median is 10.0
Cut off used is 10


In [14]:
papers['target'] = papers['num_of_citations'].apply(lambda x: True if x >= CUT_OFF else False)

In [15]:
papers.head()

,paper_title,num_of_citations,conference,year,is_link,target
0,Value ordering for finding all solutions,44,IJCAI,2005,True,True
1,A Conceptual Graph Approach for the Generation...,31,IJCAI,2007,True,True
2,Exploiting image contents in web search,34,IJCAI,2007,True,True
3,Best-First Utility-Guided Search,31,IJCAI,2007,True,True
4,Ambiguous part-of-speech tagging for improving...,30,IJCAI,2007,True,True


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(papers['paper_title'], papers['target'], stratify=papers['target'])

In [17]:
import tensorflow as tf

In [18]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocessor(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [19]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [20]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.Recall(name='AUC')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
426/426 [==============================] - 3186s 7s/step - loss: 0.7068 - accuracy: 0.5168 - precision: 0.5323 - recall: 0.6023 - AUC: 0.6023
Epoch 2/10
426/426 [==============================] - 3089s 7s/step - loss: 0.7030 - accuracy: 0.5254 - precision: 0.5399 - recall: 0.6069 - AUC: 0.6069
Epoch 3/10
426/426 [==============================] - 3059s 7s/step - loss: 0.6957 - accuracy: 0.5332 - precision: 0.5463 - recall: 0.6173 - AUC: 0.6173
Epoch 4/10
426/426 [==============================] - 3056s 7s/step - loss: 0.6941 - accuracy: 0.5384 - precision: 0.5516 - recall: 0.6127 - AUC: 0.6127
Epoch 5/10
426/426 [==============================] - 3053s 7s/step - loss: 0.6932 - accuracy: 0.5460 - precision: 0.5580 - recall: 0.6221 - AUC: 0.6221
Epoch 6/10
426/426 [==============================] - 16021s 38s/step - loss: 0.6896 - accuracy: 0.5492 - precision: 0.5609 - recall: 0.6240 - AUC: 0.6240
Epoch 7/10
426/426 [==============================] - 2960s 7s/step - loss: 0.68

In [ ]:
model.evaluate(X_test, y_test)